# Decrypting Substitution Cipher using Genetic Algorithm
 > Soroosh Sadeghian

##  Intro

### Substitution Cipher

One of the simple (and vulnerable) algorithms used for encoding texts is the substitution cipher. This cypher maps each letter of the alphabet to another letter (creating the 'key' or the 'ciphered alphabet') and substitutes each letter in the original text for its respectively mapped one.
The substitution cipher key has a very big state space (as there are 26 letters in the English alphabet, there are $26! \approx 2^{88}$ possible keys); nevertheless, this cipher is not as secure as it seems. If there is a relatively big database of words (called a dictionary) containing the majority of the words used in the ciphered text, the correctness of a certain key can be measured by using it on the ciphered text and finding meaningful words. As a result, the key can be modified and tested again until the correct key is found.
One of the ways of finding the key to a ciphered text (given a dictionary) is using the genetic algorithm, which is explained below.

### Genetic Algorithm

The genetic algorithm is based on Darwin's theory of evolution and is inspired by the process of natural selection. The process of natural selection starts with the selection of the fittest individuals from a population. They produce offspring which inherit the characteristics of the parents and will be added to the next generation. If parents have a better fitness, their offspring will be better than the parents and have a better chance at surviving. This process keeps on iterating and at the end, a generation with the fittest individuals will be found. The genetic algorithms replicate this process, first creating an initial population (which is created randomly in order to make sure individuals from the whole state space have a chance to be in the population) and then performing the natural selection on the population and it's following generations until a desirable solution is found. In order to be able to rate individuals (the units from which a population is created), a fitness function should be defined which can assign a fitness score to each of them. In addition, the algorithm should have the ability of producing new generations of the population, so crossover and mutation functions are needed. The crossover function is used to generate new offspring from the individuals and the mutation function mutates the offspring (changes them, usually randomly and with a given probability) before being added to the population. Another point to be noticed is the population size which is constant, and for the new offspring to be added to the population, the weakest individuals of the previous generation should be removed.

## Definitions in this assignment

### Individual (Chromosome)

The individuals in this assignment are lists of characters with 26 elements (a ciphered alphabet). Each alphabet letter is used exactly once in each individual and its index in the list, shows the index of the letter from the original alphabet which is mapped to it.
As an Example, consider the key __'hijazdwvupqrbcegfmnokltsxy'__, this key shows that the letter 'a' in the encoded text, means the letter 'h' in the original one; 'b' shows 'i' and so on.

### Population

The population is a list consisting of a number of the individuals. (The default size of the population is 200 in this assignment, which was obtained using trial and error)

The initial population is created using the following functions, by randomly choosing $n$ permutations of the english alphabet, with $n$ being the population size:

    def get_alphabet_list():
        return list(string.ascii_lowercase)

    def create_random_chromosome(cls):
        chromosome = []
        alphabet = cls.get_alphabet_list()
        alpha_count = len(alphabet)
        for i in range(alpha_count):
            chromosome.append(random.choice(alphabet))
            alphabet.remove(chromosome[-1])
        return tuple(chromosome)
       
    def create_initial_population(self) -> list:
        return [self.create_random_chromosome() for i in range(self._population_size)]

### Fitness Function and Dictionary

The dictionary is built using the following function:

    def build_clean_dictionary(file_path: str = './global_text.txt') -> dict:
        with open(file_path, 'r') as f:
            original_text = f.read()
        cleaned_text = re.split('\W+', original_text)
        dictionary = defaultdict(set)

        for word in cleaned_text:
            if len(word) > 1:
                dictionary[len(word)].add(word.lower())
        return dictionary

The text is split word by word, and put into a dictionary separated by their lengths (Non letter or number characters are ignored and words with a length of 1 are not considered.)

    Sample of the dictionary:
        {
            3: {'jxd', 'nor', 'all', 'bbs', 'was',...}, 
            5: {'angle', 'exert', 'toget', 'cease',...},
            13: {'influencedthe', 'wouldactually', 'independently',...},
            ...
        }

The fitness function assigns a fitness value to each individual. The fitness function used in this assignment, initializes the fitness value to 0, decodes the encoded text word by word using the key, and if the decoded word is a meaningful one according to the dictionary, it's length is added to the fitness value. The function implementation is shown below:

    def decode_word(word, chromosome):
        result = ''
        for i in word:
            if i.isalpha():
                result += chromosome[ord(i) - ord('a')]
            else:
                result += i
        return result

    def calculate_fitness_score(cls, text, chromosome, dictionary):
        fitness_score = 0
        for word in text:
            decoded_word = cls.decode_word(word, chromosome)
            if decoded_word in dictionary[len(word)]:
                fitness_score += len(word)
        return fitness_score

Although an important point is that each calculated value is cached and if a repeated individual is generated, it's fitness value is retrieved from the cache and it's not calculated again.

### Generating Offspring

Generating offspring consists of two phases, crossover and mutation which are explained below.

### Crossover

The crossover function used is the *Davis’ Order Crossover (OX1)*.

In this function, 2 crossover points are chosen randomly, and the segment between them is copied from the first parent to the first offspring, and then starting from the second crossover point in the second parent, the remaining unused numbers from the second parent is copied to the first child, wrapping around the list. Finally, the same process is repeated for the second child with the parents' roles reversed.

    def crossover(player_1, player_2):
        chromosome_length = len(player_1)
        crossover_point_1 = random.randint(0, chromosome_length - 2)
        crossover_point_2 = random.randint(
            crossover_point_1 + 1, chromosome_length - 1) + 1
        child_1 = [0 for _ in range(chromosome_length)]
        child_2 = [0 for _ in range(chromosome_length)]
        child_1[crossover_point_1:crossover_point_2] = \
            player_1[crossover_point_1:crossover_point_2]
        child_2[crossover_point_1:crossover_point_2] = \
            player_2[crossover_point_1:crossover_point_2]
        i = crossover_point_2 % chromosome_length
        j = i
        while j > crossover_point_2 - 1 or j < crossover_point_1:
            if player_2[i] not in child_1:
                child_1[j] = player_2[i]
                j = (j + 1) % chromosome_length
            i = (i + 1) % chromosome_length
        i = crossover_point_2 % chromosome_length
        j = i
        while j > crossover_point_2 - 1 or j < crossover_point_1:
            if player_1[i] not in child_2:
                child_2[j] = player_1[i]
                j = (j + 1) % chromosome_length
            i = (i + 1) % chromosome_length
        return [child_1, child_2]
        
### Mutation 

Mutation happens when new offspring is generated.

The mutation function changes a number ($n$) of genes in the chromosome with a given probability ($P_n$ is the probabilty of $n$ genes getting mutated: $P_0: 50$%, $P_1: 30$%, $P_2: 15$%, $P_3: 5$%). 

The mutation process chooses 2 random genes and switches them. This process is repeated $n$ times.

    def mutate(_chromosome):
        mutation_distribution = [0] * 50 + [1] * 30 + [2] * 15 + [3] * 5
        chromosome = _chromosome
        genes_to_mutate = random.choice(mutation_distribution)
        already_mutated_genes = []
        for i in range(genes_to_mutate):
            first_gene = random.randint(0, len(chromosome) - 1)
            while first_gene in already_mutated_genes:
                first_gene = random.randint(0, len(chromosome) - 1)
            already_mutated_genes.append(first_gene)
            second_gene = random.randint(0, len(chromosome) - 1)
            while second_gene in already_mutated_genes:
                second_gene = random.randint(0, len(chromosome) - 1)
            already_mutated_genes.append(second_gene)
            chromosome[first_gene], chromosome[second_gene] = \
                chromosome[second_gene], chromosome[first_gene]
        return chromosome
        
### Creating a new generation

While creating a new generation, the top 10% of the previous generation are directly transferred to the new generation, and then new offspring are generated by mating the top 20% of the previous generation. Offspring are generated and added to the new generation until the size of the new generation is equal to the population size. (All of the numbers are obtained through Trial and Error.)

### Solving the problem

In the process of finding the key to the cipher, a set of random individuals are created and then based on their fitness scores, new offspring are generated and this process is repeated until the fitness score of an individual passes the threshold, which is equal to 99% of the sum of the lengths of the encoded words.

As this process relies heavily on the initial random individuals, a premature convergence may occur. That is the chromosomes (individuals) may converge to a similar set excluding the valid solution, and as a result, they can't generate new and different individuals through mating and the algorithm gets stuck.

The solution used in this assignment is that if 30 generations pass without making any progress in finding the key, the population has probably converged prematurely, so the whole process restarts with a new random population to avoid getting stuck. 

Another possible solution to prevent early convergence was to increase the mutation rate (at least in the early stages) to increase randomness in the individuals and make the population more diverse, but after trying it, its effect was not as good as restarting the process, and even sometimes prevented the process from staying still and restarting, but progressed with such a slow rate it took more time to finish as opposed to restarting the process.

    def find_key(self, text, _population, reference):
        text = re.split('\W+', text.lower())
        text = list(filter(lambda word: len(word) > 1, text))
        population = _population
        threshold = sum(list(map(len, text))) * 99 // 100
        highest_fitness = 0
        already_calculated = {}
        generations = 0
        not_changed = 0
        while highest_fitness < threshold:
            if not_changed > 30:
                highest_fitness = 0
                not_changed = 0
                population = self.create_initial_population()
            generations += 1
            scores = self.calculate_fitness_scores(
                text, population, reference, already_calculated)
            for i in range(len(scores)):
                already_calculated[population[i]] = scores[i]
            sorted_indices = list(reversed(np.argsort(scores)))
            if scores[sorted_indices[0]] > highest_fitness:
                highest_fitness = scores[sorted_indices[0]]
                not_changed = 0
            else:
                not_changed += 1
            sorted_population = [population[i] for i in sorted_indices]
            new_generation = [
                *sorted_population[:len(sorted_population) // 10]]
            while len(new_generation) < len(sorted_population):
                first_chromosome = random.choice(
                    sorted_population[:len(sorted_population) // 5])
                second_chromosome = random.choice(
                    sorted_population[:len(sorted_population) // 5])
                children = self.generate_offspring(
                    first_chromosome, second_chromosome)
                for i in children:
                    if len(new_generation) < len(population):
                        new_generation.append(i)
            population = new_generation
        return sorted_population[0]
        
### Decoding the text

After finding the key, the text is decoded by swapping each character with its mapped one.

    def create_key_dict(cls, key):
        alpha = cls.get_alphabet_list()
        key_dict = {}
        key_length = len(key)
        for i in range(key_length):
            key_dict[alpha[i]] = key[i]
            key_dict[alpha[i].upper()] = key[i].upper()
        return key_dict

    def decode_text(self, key):
        decoded_text = [key.get(char, char) for char in self._encoded_text]
        return ''.join(decoded_text)
        
### Effects of mutation and comparing it with crossover

Mutation prevents the population from premature convergence and gives it a chance to create new, different chromosomes which may lead to better results. But the more effective process is crossover as it tries to generate offspring from the fittest individuals, which usually leads to even fitter individuals.

### Increasing the population in each iteration

Increasing the population in each iteration quickly leads to very big population size which is not desirable as calculating the fitness of such a big population is a very time-consuming process and leads to the whole algorithm becoming slower.

## *Decoder*  Class Code

In [25]:
import re
import string
import random
from collections import defaultdict
import numpy as np
from functools import reduce
import time


DEFAULT_POPULATION_SIZE = 200
ELEMENT_BY_ELEMENT = 1
SINGLE_POINT_CROSSOVER = 2
MULTIPOINT_CROSSOVER = 3


class Decoder:
    _encoded_text: str = None
    _population_size = None

    def __init__(self, text: str, population_size: int = DEFAULT_POPULATION_SIZE):
        self._population_size = population_size
        self._encoded_text = text

    def create_initial_population(self) -> list:
        return [self.create_random_chromosome() for i in range(self._population_size)]

    @staticmethod
    def get_alphabet_list():
        return list(string.ascii_lowercase)

    @classmethod
    def create_random_chromosome(cls):
        chromosome = []
        alphabet = cls.get_alphabet_list()
        alpha_count = len(alphabet)
        for i in range(alpha_count):
            chromosome.append(random.choice(alphabet))
            alphabet.remove(chromosome[-1])
        return tuple(chromosome)

    @staticmethod
    def build_clean_dictionary(file_path: str = './global_text.txt') -> dict:
        with open(file_path, 'r') as f:
            original_text = f.read()
        cleaned_text = re.split('\W+', original_text)
        dictionary = defaultdict(set)

        for word in cleaned_text:
            if len(word) > 1:
                dictionary[len(word)].add(word.lower())
        return dictionary

    @staticmethod
    def decode_word(word, chromosome):
        result = ''
        for i in word:
            if i.isalpha():
                result += chromosome[ord(i) - ord('a')]
            else:
                result += i
        return result

    @classmethod
    def calculate_fitness_score(cls, text, chromosome, dictionary):
        fitness_score = 0
        for word in text:
            decoded_word = cls.decode_word(word, chromosome)
            if decoded_word in dictionary[len(word)]:
                fitness_score += len(word)
        return fitness_score

    @classmethod
    def calculate_fitness_scores(cls, text, population, dictionary, already_calculated):
        scores = []
        for chromosome in population:
            if chromosome not in already_calculated:
                score = cls.calculate_fitness_score(text, chromosome, dictionary)
                scores.append(score)
            else:
                scores.append(already_calculated.get(chromosome))
        return scores

    @staticmethod
    def crossover(player_1, player_2):
        chromosome_length = len(player_1)
        crossover_point_1 = random.randint(0, chromosome_length - 2)
        crossover_point_2 = random.randint(crossover_point_1 + 1, chromosome_length - 1) + 1
        child_1 = [0 for _ in range(chromosome_length)]
        child_2 = [0 for _ in range(chromosome_length)]
        child_1[crossover_point_1:crossover_point_2] = player_1[crossover_point_1:crossover_point_2]
        child_2[crossover_point_1:crossover_point_2] = player_2[crossover_point_1:crossover_point_2]
        i = crossover_point_2 % chromosome_length
        j = i
        while j > crossover_point_2 - 1 or j < crossover_point_1:
            if player_2[i] not in child_1:
                child_1[j] = player_2[i]
                j = (j + 1) % chromosome_length
            i = (i + 1) % chromosome_length
        i = crossover_point_2 % chromosome_length
        j = i
        while j > crossover_point_2 - 1 or j < crossover_point_1:
            if player_1[i] not in child_2:
                child_2[j] = player_1[i]
                j = (j + 1) % chromosome_length
            i = (i + 1) % chromosome_length
        return [child_1, child_2]

    @staticmethod
    def mutate(_chromosome):
        mutation_distribution = [0] * 50 + [1] * 30 + [2] * 15 + [3] * 5
        chromosome = _chromosome
        genes_to_mutate = random.choice(mutation_distribution)
        already_mutated_genes = []
        for i in range(genes_to_mutate):
            first_gene = random.randint(0, len(chromosome) - 1)
            while first_gene in already_mutated_genes:
                first_gene = random.randint(0, len(chromosome) - 1)
            already_mutated_genes.append(first_gene)
            second_gene = random.randint(0, len(chromosome) - 1)
            while second_gene in already_mutated_genes:
                second_gene = random.randint(0, len(chromosome) - 1)
            already_mutated_genes.append(second_gene)
            chromosome[first_gene], chromosome[second_gene] = chromosome[second_gene], chromosome[first_gene]
        return chromosome
        

    @classmethod
    def generate_offspring(cls, player_1: list, player_2: list):
        children = cls.crossover(player_1, player_2)
        for i in range(len(children)):
            children[i] = tuple(cls.mutate(children[i]))
        return children

    def find_key(self, text, _population, reference):
        text = re.split('\W+', text.lower())
        text = list(filter(lambda word: len(word) > 1, text))
        population = _population
        threshold = sum(list(map(len, text))) * 99 // 100
        highest_fitness = 0
        already_calculated = {}
        generations = 0
        not_changed = 0
        while highest_fitness < threshold:
            if not_changed > 30:
                highest_fitness = 0
                not_changed = 0
                population = self.create_initial_population()
            generations += 1
            scores = self.calculate_fitness_scores(text, population, reference, already_calculated)
            for i in range(len(scores)):
                already_calculated[population[i]] = scores[i]
            sorted_indices = list(reversed(np.argsort(scores)))
            if scores[sorted_indices[0]] > highest_fitness:
                highest_fitness = scores[sorted_indices[0]]
                not_changed = 0
            else:
                not_changed += 1
            sorted_population = [population[i] for i in sorted_indices]
            new_generation = [*sorted_population[:len(sorted_population) // 10]]
            while len(new_generation) < len(sorted_population):
                first_chromosome = random.choice(
                    sorted_population[:len(sorted_population) // 5])
                second_chromosome = random.choice(
                    sorted_population[:len(sorted_population) // 5])
                children = self.generate_offspring(
                    first_chromosome, second_chromosome)
                for i in children:
                    if len(new_generation) < len(population):
                        new_generation.append(i)
            population = new_generation
        return sorted_population[0]

    @classmethod
    def create_key_dict(cls, key):
        alpha = cls.get_alphabet_list()
        key_dict = {}
        key_length = len(key)
        for i in range(key_length):
            key_dict[alpha[i]] = key[i]
            key_dict[alpha[i].upper()] = key[i].upper()
        return key_dict

    def decode_text(self, key):
        decoded_text = [key.get(char, char) for char in self._encoded_text]
        return ''.join(decoded_text)

    def decode(self):
        reference = self.build_clean_dictionary()
        text = self._encoded_text
        population = self.create_initial_population()
        start = time.time()
        key = self.find_key(text, population, reference)
        end = time.time()
        key_dict = self.create_key_dict(key)
        decoded_text = self.decode_text(key_dict)
        return decoded_text

## Average running time
As seen below, the average running time for 10 tests is 21.45 seconds

In [26]:
import time
NUMBER_OF_TESTS = 10

with open('./encoded_text.txt') as f:
    t = f.read()
d = Decoder(t)
avg_time = 0
for i in range(NUMBER_OF_TESTS):
    start = time.time()
    text = d.decode()
    avg_time += time.time() - start
avg_time /= NUMBER_OF_TESTS
print('Average running time: {}'.format(avg_time))
with open('./out.txt', 'w+') as f:
    f.write(text)

Average running time: 21.45645639896393
